# Create interannual snowDynamics products
- 1st run: with implausible snow filter (7 day buffer): is_7 (default)
- 2nd run: with implausible snow filter (30 day buffer): is_30
- 2nd run: without implausible snow filter: is_none

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import xarray as xr
import os

import sys
sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import PreProcess_Utils as pputil

sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/Modelling/Utilities')
import Snow_Utils as snow_util

In [2]:
from dask.distributed import LocalCluster

cluster = LocalCluster(n_workers = 10, threads_per_worker = 10, memory_limit = 0) # Refined through testing, removing memory_limit is important
client = cluster.get_client()
client # Can open in other window by clicking 'Dashboard' link below

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59052,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:59092,Total threads: 10
Dashboard: http://127.0.0.1:59102/status,Memory: 0 B
Nanny: tcp://127.0.0.1:59056,


In [3]:
# Calculated from IMS (4 SnowDynamics_IMS)
sstart_first = 248
send_last = 166
smin_avg = 205

In [4]:
winterYear = r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\Alaska\3 HLS snowDynamics\is_none' # winterYear # winterYear products
path = r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\Alaska\3 HLS snowDynamics\is_none\interannual' # Where to so save interannual products

ID = 'winterYear'
names = ['start', 'end', 'length', 'periods', 'status', 'start_u', 'end_u', 'length_u']

In [5]:
# Load winter year snow dynamics cubes
snowDynamics = pputil.uploadGT(winterYear, ID, type = 'snowDynamics', names = names).compute()
print('Loaded winterYear snowDynamics dataset (' + str(len(snowDynamics)) + ' variables, ' + str(snowDynamics.sizes['winterYear']) + ' winterYears).')

# Identify implausible snow period
#implausible_snow = [send_last + 7, sstart_first - 7] # Week buffer
#implausible_snow = [send_last + 30, sstart_first - 30] # Month buffer
implausible_snow = [] # No implausible snow
            
# date1 = dt.datetime(2023, 12, 31) + dt.timedelta(implausible_snow[0])
# date1 = str(date1.month) + '/' + str(date1.day)
    
# date2 = dt.datetime(2023, 12, 31) + dt.timedelta(implausible_snow[1])
# date2 = str(date2.month) + '/' + str(date2.day)
            
# print('Implausible snow period identified as ' + date1 + ' (doy ' + str(implausible_snow[0]) + ') to ' + date2 + ' (doy ' + str(implausible_snow[1]) + ').')

# Create interannual snowDynamics dataset using weighted mean
snowDynamics = snow_util.interannualSnowDynamics(snowDynamics, 
                                                 min_count = 'half',
                                                 products = ['start', 'end', 'length', 'periods', 'status'],
                                                 uncertainty = ['start_u', 'end_u', 'length_u'], # Maybe do not need uncertainty means in future (use quality)
                                                 clearObsPerDay = [], # Not included
                                                 type = 'mean_weighted',
                                                 implausible_snow = implausible_snow,
                                                 sd = False, 
                                                 quality = True,
                                                 best_value = True,
                                                 verbose = False)

print('Output interannual snowDynamics products ' + str(tuple(list(snowDynamics.keys()))) + '.')

# # Save dataset to NetCDF
# pputil.downloadNC(snowDynamics, interannual_tile, filename, type = '')
# print('Interannual dataset saved: ' + filename)

# Final cleaning and save dataset to TIFs
variables = list(snowDynamics.keys()) # Name of all data variables as list 

for variable in variables:

    da = snowDynamics[variable].astype('float32') # Single snow dynamic stat from dataset (float32 supporting NAs)

    da.rio.write_nodata(-999, encoded = True, inplace = True) # Ensured NaN NoData values are saved
    da.rio.to_raster(os.path.join(path, 'HLS_Fmask_' + variable + '_1823_AlaskaSite.tif'))        

    print('Product ' + str(variables.index(variable) + 1) + ' of ' + str(len(variables)) + ' saved: HLS_Fmask_' + variable + '_1823_AlaskaSite.tif')

Loaded winterYear snowDynamics dataset (8 variables, 5 winterYears).
Output interannual snowDynamics products ('snow_start_mn', 'snow_end_mn', 'snow_length_mn', 'snow_periods_mn', 'pPerennialSnow', 'pSnowFree', 'snow_start_u_mn', 'snow_start_q_mn', 'snow_start_bv', 'snow_start_bvy', 'snow_start_bvq', 'snow_end_u_mn', 'snow_end_q_mn', 'snow_end_bv', 'snow_end_bvy', 'snow_end_bvq', 'snow_length_u_mn', 'snow_length_q_mn', 'snow_length_bv', 'snow_length_bvy', 'snow_length_bvq').
Product 1 of 21 saved: HLS_Fmask_snow_start_mn_1823_AlaskaSite.tif
Product 2 of 21 saved: HLS_Fmask_snow_end_mn_1823_AlaskaSite.tif
Product 3 of 21 saved: HLS_Fmask_snow_length_mn_1823_AlaskaSite.tif
Product 4 of 21 saved: HLS_Fmask_snow_periods_mn_1823_AlaskaSite.tif
Product 5 of 21 saved: HLS_Fmask_pPerennialSnow_1823_AlaskaSite.tif
Product 6 of 21 saved: HLS_Fmask_pSnowFree_1823_AlaskaSite.tif
Product 7 of 21 saved: HLS_Fmask_snow_start_u_mn_1823_AlaskaSite.tif
Product 8 of 21 saved: HLS_Fmask_snow_start_q_mn_18

In [6]:
# Memory management at end
del snowDynamics

# Testing

In [7]:
# import glob
# tifs = glob.glob(os.path.join(winterYear, ID + '/**/*tif'), recursive = True)
# tifs

In [8]:
# import rioxarray as rxr
# first = rxr.open_rasterio(tifs[0])
# first

In [9]:
# cube = xr.Dataset(coords = dict(x = ('x', first['x'].values), y = ('y', first['y'].values)))
# cube.rio.write_crs(first.rio.crs, inplace = True)
# cube

In [10]:
# import re
# num4 = [] # 4 digits (winterYears)
# for tif in tifs: # All tif filenames
#     num4.append(re.findall(r'\d{4}', tif)[0]) # First set of 4 numbers in all filenames
# num4 = list(dict.fromkeys(num4)) # Ordered unique sets of 4 numbers
# num4

In [11]:
# winter = ['20' + num4[i][0:2] + '-20' + num4[i][2:4] for i in range(len(num4))] # Convert to proper convention
# winter = xr.Variable('winterYear', winter) # Set as variable
# winter

In [12]:
# tifs[0]

In [13]:
# stat = 'end'
# for tif in tifs:
#     if (stat in tif) & (not stat + '_u' in tif):
#         print(tif)

In [14]:
# rxr.open_rasterio(tifs[1], chunks = 'auto')

In [15]:
# for stat in names:
#     cube['snow_' + stat] = xr.concat([rxr.open_rasterio(tif, chunks = 'auto') for tif in tifs if stat in tif], dim = winter).squeeze()

In [16]:
# snowDynamics

In [17]:
# cube['snow_' + stat] = xr.concat([rxr.open_rasterio(tif, chunks = 'auto') for tif in tifs if stat in tif], dim = winter).squeeze()

In [18]:
# cube

In [19]:
# cube['snow_end_u'][4].plot()

In [20]:
# %%time
# snowDynamics = pputil.uploadGT(winterYear, ID, type = 'snowDynamics', names = names)
# snowDynamics

In [21]:
# import rioxarray as rxr
# #path = r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\Alaska\3 HLS snowDynamics\is_7\winterYear\1819\HLS_Fmask_snow_end_winterYear1819_AlaskaSite.tif"
# #path = r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\HBL\TemporalSnow\Fmask_V1\Products2\winterYear_roi\1819\HLS_Fmask_snow_end_winterYear1819_HBL3x3.tif"
# #path = r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\Alaska\3 HLS snowDynamics\is_7\interannual\HLS_Fmask_snow_end_mn_1823_AlaskaSite.tif"
# #path = r"F:\HLS\HBL\Products\SnowDynamics\winterYear_roi\1819\HLS_Fmask_snow_end_winterYear1819_HBL.tif"
# path = r"F:\HLS\HBL\Products\SnowDynamics\interannual_roi\HLS_Fmask_snow_end_mn_1823_HBL.tif"
# test = rxr.open_rasterio(path, mask_and_scale = True)
# test

In [22]:
# import numpy as np
# np.unique(test.values, return_counts = True)

In [23]:
# test.isnull().sum()